In [12]:
# importing packages and setting constants

import pandas as pd
import numpy as np
import astropy.constants as const
from astropy.table import Table
from scipy import integrate
import csv

h=6.626196*10**(-27)
c=2.997924562*10**(10)
k=1.380622*10**(-16)
x=h*c/k
Tcmb=3

In [ ]:
# making dataframe from energy level data file

fp='/Users/hannahgruber/Documents/ctex scripts/dnc_data.csv'
dncfile = pd.read_csv(fp,header=None)
dncfile

In [14]:
# read in important quantities from this dataframe

nu_21=dncfile.iloc[:-1,0].to_numpy()
g_u=dncfile.iloc[:-1,5].to_numpy()
E_u=dncfile.iloc[1:,4].to_numpy()*x
# A_21=float(input("Einstein A:"))
print(g_u)

g_2=g_u[0]
E_2=E_u[0]

[ 3  5  7  9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47 49
 51 53 55 57 59 61 63 65 67 69 71 73 75 77]


In [4]:
'''
create arrays for taupk and dv
sample small grid below

taupk=np.arange(.001,.02,.009)
dv=np.arange(.3,.4,.1)
'''

# i,j=np.meshgrid(taupk,dv) # using numpy technique to create a grid... 
# pretty sure this creates 2 arrays that have the list of points
# i would be the taupk coords, written as many times as i need to match it with each dv in j

3

In [15]:
taupk=np.logspace(.001,10,10000)
dv=np.logspace(.15,.99,10000)

In [17]:
# reads in intensity and velocity offsets for hyperfine lines and creates range of v

fp2='/Users/hannahgruber/Documents/ctex scripts/dnc_vR.csv'
df2=pd.read_csv(fp2,header=None)
df2

Ri=df2.iloc[:,1].to_numpy()
vi=df2.iloc[:,0].to_numpy()

for delv in dv:
    sigma_v=dv/((8*np.log(2))**(1/2))
vmin=min(vi)-(3*sigma_v)
vmax=max(vi)+(3*sigma_v)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/hannahgruber/Documents/ctex scripts/dnc_vR.csv'

In [6]:
sigma_v=dv/((8*np.log(2))**(1/2))
vmin=min(vi)-(3*sigma_v)
vmax=max(vi)+(3*sigma_v)

In [7]:
# calculate equivalent width based on velocity and taupk

def W_v():
    # W_v=[]
    z=[]
    for delv in dv:
        sigma_v=delv/((8*np.log(2))**(1/2))
        vmin=min(vi)-(4*sigma_v)
        vmax=max(vi)+(4*sigma_v)
        v=np.arange(vmin,vmax,.01)
        for t in taupk:
            y=[]
            for vel in v:
                tau=t*np.sum(Ri*np.exp(-(4*np.log(2))*((vel-vi)**2)/(delv**2)))
                y.append(1-np.exp(-tau))
            # W_v.append(integrate.simpson(y,x=v,axis=0))
            W_v=integrate.simpson(y,x=v,axis=0)
            z.append([delv,t,W_v])
    return z
W_v()

[[0.3, 0.001, 0.0003192950726934102],
 [0.3, 0.009999999999999998, 0.003189068284090988],
 [0.3, 0.018999999999999996, 0.006051867157170733],
 [0.4, 0.001, 0.0004257139164271087],
 [0.4, 0.009999999999999998, 0.004250798070275657],
 [0.4, 0.018999999999999996, 0.008064496392875407]]

In [8]:
# makes equiv width into dataframe, and then into a csv file

data=W_v()
df_W=pd.DataFrame(data,columns=["dv","taupk","W_v"])
df_W

df_W.to_csv("W_v_table.csv", index=False)